In [317]:
import seaborn as sns
import numpy as np
import pandas as pd
import snscrape.modules.twitter as sntwitter

sns.set(rc = {'figure.figsize':(15, 7)})
sns.set_style('white')
pd.set_option('display.max_columns', 500)

In [318]:
tweets1 = pd.read_csv('./data/Tweet.csv')
company1 = pd.read_csv('./data/Company_Tweet.csv')
stocks1 = pd.read_csv('./data/stock_data_2010_2020.csv')

tweets2 = pd.read_csv('./data/stock_tweets_2021_2022.csv', parse_dates=['Date'])
stocks2 = pd.read_csv('./data/stock_data_2021_2022.csv')

tweets1.head()

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,1420070510,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0
3,550442977802207232,ShowDreamCar,1420070807,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
4,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1


### Scraping Twitter

In [217]:
tweets2['Stock Name'].unique()

array(['TSLA', 'MSFT', 'PG', 'META', 'AMZN', 'GOOG', 'AMD', 'AAPL',
       'NFLX', 'TSM', 'KO', 'F', 'COST', 'DIS', 'VZ', 'CRM', 'INTC', 'BA',
       'BX', 'NOC', 'PYPL', 'ENPH', 'NIO', 'ZS', 'XPEV'], dtype=object)

In [212]:
company1['ticker_symbol'].unique()

array(['AAPL', 'GOOG', 'GOOGL', 'AMZN', 'MSFT', 'TSLA'], dtype=object)

In [241]:
lst_to_scrape = ['PG', 'META', 'AMD', 'NFLX', 'TSM', 'KO', 'F', 'COST', 'DIS', 'VZ', 
             'CRM', 'INTC', 'BA', 'BX', 'NOC', 'PYPL', 'ENPH', 'NIO', 'ZS', 'XPEV']

In [263]:
twt_list = []
twt_list2 = []

for ticker in lst_to_scrape:
    for idx, tweet in enumerate(sntwitter.TwitterSearchScraper(
        f'@{ticker} since:2015-01-31 until:2019-12-31').get_items()):
        
        if idx > 15000:
            break

        twt_list.append([tweet.date, tweet.rawContent, ticker])
        
for ticker in tweets2['Stock Name'].unique():
    for idx, tweet in enumerate(sntwitter.TwitterSearchScraper(
        f'@{ticker} since:2020-01-01 until:2021-09-29').get_items()):
        
        if idx > 15000:
            break

        twt_list2.append([tweet.date, tweet.rawContent, ticker])

In [248]:
df = pd.DataFrame(twt_list, columns=['Date', 'Tweet', 'Stock Name']).merge(
    tweets[['Stock Name', 'Company Name']].drop_duplicates(),
    on='Stock Name')

df2 = pd.DataFrame(twt_list2, columns=['Date', 'Tweet', 'Stock Name']).merge(
    tweets[['Stock Name', 'Company Name']].drop_duplicates(),
    on='Stock Name')

In [250]:
df.to_csv('./data/scraped_tweets_2015_2019.csv', index=False)
df2.to_csv('./data/scraped_tweets_2020_2021.csv', index=False)

### Scraping Yahoo Finances

In [319]:
from yahoo_fin.stock_info import get_analysts_info
import yahoo_fin.stock_info as si

In [320]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'

In [336]:
df_list = []

for ticker in tweets2['Stock Name'].unique():
    try:
        data = si.get_data(ticker, start_date = '01/01/2015', end_date = '09/29/2022').reset_index()
        data.columns = stocks2.columns

        globals()[ticker] = data
        df_list.append(globals()[ticker])
    except:
        data = si.get_data(ticker, start_date = '01/01/2020', end_date = '09/29/2021').reset_index()
        data.columns = stocks2.columns

        globals()[ticker] = data
        df_list2.append(globals()[ticker])

In [339]:
df = pd.concat(df_list)
df.to_csv('./data/scraped_stock_2015_2022.csv')

In [340]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Stock Name
0,2015-01-02,14.858000,14.883333,14.217333,14.620667,14.620667,71466000,TSLA
1,2015-01-05,14.303333,14.433333,13.810667,14.006000,14.006000,80527500,TSLA
2,2015-01-06,14.004000,14.280000,13.614000,14.085333,14.085333,93928500,TSLA
3,2015-01-07,14.223333,14.318667,13.985333,14.063333,14.063333,44526000,TSLA
4,2015-01-08,14.187333,14.253333,14.000667,14.041333,14.041333,51637500,TSLA
